# ML model – klasifikace kategorie článků

Data: vlastní crawl z idnes.cz, novinky.cz, aktualne.cz, irozhlas.cz  
Postup: načtení → předzpracování → trénování → vyhodnocení

## 1. Načtení dat

Cesta k datům: buď `../crawl/articles_part1.jsonl`, nebo po nahrání na Colab disk.

In [ ]:
import json

DATA_PATH = "../crawl/articles_part1.jsonl"  # lokálně; na Colabu změň na cestu k souboru
MAX_ROWS = 5000  # pro rychlý běh; pro finální model můžeš zvýšit nebo použít víc

rows = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= MAX_ROWS:
            break
        line = line.strip()
        if not line:
            continue
        try:
            rows.append(json.loads(line))
        except json.JSONDecodeError:
            continue

print(f"Načteno záznamů: {len(rows)}")